# 큐 러닝
- 큐 러닝(Q-learning) : 주어진 상태에서 행동을 취했을 경우 받을 수 있는 보상의 기댓값을 예측하는 큐함수를 사용하여 최적화된 정책을 학습하는 강화학습 기법. 즉 여러 실험을 반복해 최적의 정책을 학습

과정 : 
1. 초기화 : 큐텡이블에 있는 모든 큐 값을 0으로 초기화
2. 행동 a를 선택하고 실행
3. 보상 r과 다음 상태 s'을 관찰
4. 상태 s'에서 가능한 모든 행동에 대해 가장 높은 큐 값을 갖는 행동인 a' 선택
5. 공식을 이용하여 상태에 대한 큐 값을 업데이트
6. 종료 상태에 도달할 때 까지 2~5 반복

# 딥 큐 러닝
- 합성곱 신경망을 이용하여 큐 함수를 학습하는 강화 학습 기법. 합성곱층을 깊게 하여 훈련할 때, 큐 값의 정확도를 높이는 것을 목표로 함

- 합성곱 신경망을 활용한 큐 함수
게임에서 택시는 랜덤한 위치에서 시작하고 승객은 랜덤한 위치 (R,G,B,Y)에 답승, 기사는 승객 위치로 이동하여 승객을 태우고 목적지까지 이동, 목적지는 승객이 출발한 위치를 제외한 나머지 위치중 하나가 됨. 승객이 목적지에서 내리면 게임끝

- 승객 위치 표현 : 
R:0, G:1, Y:2, B:3, 4:택시안
- 승객의 현재 위치 및 목적지에 대한 정보
R:0, G:1, Y:2, B:3
- 에이전트가 취할 수 있는 행동
0:아래, 1:위, 2:오른족, 3:왼쪽, 4:승객 태우기, 5:승객 내리기

In [2]:
!pip install progressbar2
!pip install gym # 강화학습 환경을 구성할 수 있도록 도와주는 파이썬 패키지

  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616828 sha256=8792431acf1c51b1f6d34308fc02eb8ced816685dbf1fa1d291585702b636b05
  Stored in directory: c:\users\이신행\appdata\local\pip\cache\wheels\27\6d\b3\a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
Successfully built gym


In [3]:
#라이브러리 호출
import numpy as np
import random
from IPython.display import clear_output # 에이전트가 환경에 반응한 경험을 저장
from collections import deque 
import gym
from tensorflow.keras import Model,Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape
from tensorflow.keras.optimizers import Adam

In [4]:
#한글깨짐 해결
import matplotlib as mpl
import matplotlib.pylab as plt
from matplotlib import font_manager

font_fname = 'C:/Windows/Fonts/malgun.ttf'
font_family = font_manager.FontProperties(fname=font_fname).get_name()

plt.rcParams["font.family"] = font_family

In [5]:
# Taxi-v3에 대한 환경
env=gym.make('Taxi-v3').env # Taxi-v3 객체를 인스턴스로 생성
env.render() # 환경 및 에이전트의 현재 상태를 render 메서드를 사용하여 표시

print('취할 수 있는 상태 수 : {}'.format(env.observation_space.n)) # 환경의 모든 상태에 대해 접근
print('취할 수 있는 행동 수 : {}'.format(env.action_space.n)) # 환경의 모든 행동에 접근

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

취할 수 있는 상태 수 : 500
취할 수 있는 행동 수 : 6


In [6]:
# 에이전트 구현
class Agent:
    def __init__(self, env, optimizer): # 상태와 행동을 초기화
        self._state_size = env.observation_space.n # 환경 속성 observation_space 초기화
        self._action_size = env.action_space.n # 환경 속성 action_space 초기화
        self._optimizer = optimizer # 옵티마이저 초기화
        self.expirience_replay = deque(maxlen=2000) # 과거 행동에 대한 기억 초기화
        
        self.gamma = 0.6 # 할인율 초기화
        self.epsilon = 0.1 # 탐험 비율 초기화

        self.q_network = self.build_compile() # build_compile() 함수를 사용하여 큐 네트워크 생성
        self.target_network = self.build_compile() # build_compile() 함수를 사용하여 타깃 큐-네트워크 생성 
        self.target_model()

    def store(self, state, action, reward, next_state, terminated):
        self.expirience_replay.append((state, action, reward, next_state, terminated))

    def build_compile(self): # 네트워크 구성을 위한 함수
        model = Sequential()
        model.add(Embedding(self._state_size, 10, input_length=1)) # 1
        model.add(Reshape((10,)))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(self._action_size, activation='linear'))
        model.compile(loss='mse', optimizer=self._optimizer)
        return model

    def target_model(self): # 가중치를 적용하기 위한 함수
        self.target_network.set_weights(self.q_network.get_weights())

    def act(self, state): # 탐험을 위한 함수
        if np.random.rand() <= self.epsilon: # 2
            return env.action_space.sample()
        q_values = self.q_network.predict(state)
        return np.argmax(q_values[0])

    def retrain(self, batch_size): # 큐 네트워크 훈련에 대한 함수
        minibatch = random.sample(self.expirience_replay, batch_size) # 리플레이 메모리에서 랜덤한 데이터 선택
        for state, action, reward, next_state, terminated in minibatch:
            target = self.q_network.predict(state)
            if terminated:
                target[0][action] = reward
            else:
                t = self.target_network.predict(next_state)
                target[0][action] = reward + self.gamma * np.amax(t)
            self.q_network.fit(state, target, epochs=1, verbose=0) # 큐 네트워크 훈련

- 1 : 네트워크의 첫번째 임베딩에 등장, 여기서 임베딩 역할은 가능한 상태 500개를 값 열개로 표현하는 것
    - self._state_size : 입력에 대한 차원(총 입력 개수), 500개
    - 10 : 출력에 대한 차원(결과로 나오는 임베딩 벡터의 크기)로 10
    - input_length : 입력 시퀀스의 길이를 의미
- 2 : 택시기사가 승객을 태우고 목적지로 이동하기 위한 탐험을 결정하는 방법. 여기서는 랜덤으로 값을 선택한 후 행동을 취하는 방법

In [11]:
# 훈련 준비
optimizer = Adam(learning_rate=0.01)
agent = Agent(env, optimizer)
batch_size = 32
num_of_episodes = 10
timesteps_per_episode = 10
agent.q_network.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1, 10)             5000      
                                                                 
 reshape_4 (Reshape)         (None, 10)                0         
                                                                 
 dense_16 (Dense)            (None, 50)                550       
                                                                 
 dense_17 (Dense)            (None, 50)                2550      
                                                                 
 dense_18 (Dense)            (None, 50)                2550      
                                                                 
 dense_19 (Dense)            (None, 6)                 306       
                                                                 
Total params: 10,956
Trainable params: 10,956
Non-trai

In [12]:
# 모델 훈련
for e in range(0, num_of_episodes):
    state = env.reset() # 환경 재설정
    state = np.reshape(state, [1, 1])

    reward = 0 # 보상 변수 초기화
    terminated = False

    for timestep in range(timesteps_per_episode):
        action = agent.act(state) # act()함수 실행
        next_state, reward, terminated, info = env.step(action) # 에이전트가 단계별 행동 취함
        next_state = np.reshape(next_state, [1, 1])
        agent.store(state, action, reward, next_state, terminated)
        state = next_state

        if terminated:
            agent.target_model()
            break

        if len(agent.expirience_replay) > batch_size:
            agent.retrain(batch_size)
        
    if (e + 1) % 10 == 0:
        print("**********************************")
        print("Episode: {}".format(e + 1))
        env.render()
        print("**********************************")

**********************************
Episode: 10
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
**********************************


# 몬테카를로 트리 탐색
- 모든 트리 노드를 대상으로 탐색하는 대신, 게임 시뮬레이션을 이용하여 가장 가능성이 높아보이는 방향으로 행동을 결정하는 탐색 방법. 즉 경우의 수가 많을 때 순차적으로 시도하는 것이 아닌 무작위 방법 중 가장 승률이 높은 값을 기반으로 시도하는 것

과정
1. 선택 : 루트 r에서 시작하여 현재까지 펼쳐진 트리 중 가장 승산 있는 자식 노드 l을 선택
2. 확장 : 노드 l에서 게임이 종료되지 않는다면, 그 이상의 자식 노드를 생성하고, 그 중 하나의 노드 c를 선택
3. 시뮬레이션 : 노드 c에서 랜덤으로 자식 노드를 선택하여 게임 반복 진행
4. 역전파 : 시뮬레이션 결과로 c,l,r까지 경로에 있는 노드들의 정보 갱신

## 틱택토 게임 구현
- 커맨드 라인에서 게임을 진행할 수 있는 2인용 틱택토 게임 만들 예정. 
- 처음에는 비어있는 보드를 만든 후 플레이어에게서 입력을 받아 승리 조건을 확인하고 승자를 선언하거나, 전체 보드가 채워졌음에도 아무도 이기지 않으면 결과를 동점으로 선언
- 보드는 키보드의 숫자 패드처럼 번호가 써 있음. 플레이어가 보드판에 숫자를 가로세로 혹은 대각선으로 채우면 승리

- key : 보드의 블록, value : 플레이어의 움직임

In [14]:
# 보드 생성
boarder = {'1': ' ' , '2': ' ' , '3': ' ' ,
            '4': ' ' , '5': ' ' , '6': ' ' ,
            '7': ' ' , '8': ' ' , '9': ' ' }

board_keys=[]
for key in boarder:
    board_keys.append(key)

In [15]:
# 화면 출력 함수 정의
def visual_Board(board_num): # 1
    print(board_num['1'] + '|' + board_num['2'] + '|' + board_num['3'])
    print('-+-+-')
    print(board_num['4'] + '|' + board_num['5'] + '|' + board_num['6'])
    print('-+-+-')
    print(board_num['7'] + '|' + board_num['8'] + '|' + board_num['9'])

- 1 : 게임을 진행할 때마다 업데이트된 보드를 출려해야 하기 때문에 visual_board()함수를 호출하여 매번 쉽게 보드를 출력할 수 있도록 함

In [ ]:
#### 보드 이동 함수 정의
def game(): # 플레이어의 보드 이동을 위한 함수
    turn = 'X'
    count = 0
    
    for i in range(8):
        visual_Board(boarder)
        print("당신 차례입니다," + turn + ". 어디로 이동할까요?")
        move = input()        
        if boarder[move] == ' ':
            boarder[move] = turn
            count += 1
        else:
            print("이미 채워져있습니다.\n어디로 이동할까요?")
            continue

        if count >= 5: # 플레이어 x또는 y가 다섯번 이동후 이겼는지 확인
            if boarder['1'] == boarder['2'] == boarder['3'] != ' ': # 상단 1,2,3이 채워졌을 경우
                visual_Board(boarder)
                print("\n게임 종료.\n")                
                print(" ---------- " +turn + "가 승리했습니다. -----------")               
                break

            elif boarder['4'] == boarder['5'] == boarder['6'] != ' ': # 중단 4,5,6이 채워졌을 경우
                visual_Board(boarder)
                print("\n게임 종료.\n")                
                print(" ---------- " +turn + "가 승리했습니다. -----------")
                break

            elif boarder['7'] == boarder['8'] == boarder['9'] != ' ': # 하단 7,8,9가 채워졌을 경우
                visual_Board(boarder)
                print("\n게임 종료.\n")                
                print(" ---------- " +turn + "가 승리했습니다. -----------")
                break

            elif boarder['1'] == boarder['4'] == boarder['7'] != ' ': # 보드의 왼쪽 1,4,7이 채워졌을 경우
                visual_Board(boarder)
                print("\n게임 종료.\n")                
                print(" ---------- " +turn + "가 승리했습니다. -----------")
                break

            elif boarder['2'] == boarder['5'] == boarder['8'] != ' ': # 보드의 중간 2,5,8이 채워졌을 경우
                visual_Board(boarder)
                print("\n게임 종료.\n")                
                print(" ---------- " +turn + "가 승리했습니다. -----------")
                break

            elif boarder['3'] == boarder['6'] == boarder['9'] != ' ': # 보드의 오른쪽 3,6,9가 채워졌을 경우
                visual_Board(boarder)
                print("\n게임 종료.\n")                
                print(" ---------- " +turn + "가 승리했습니다. -----------")
                break 

            elif boarder['1'] == boarder['5'] == boarder['9'] != ' ':  # 대각선 1,5,9가 채워졌을 경우
                visual_Board(boarder)
                print("\n게임 종료.\n")                
                print(" ---------- " +turn + "가 승리했습니다. -----------")
                break

            elif boarder['3'] == boarder['5'] == boarder['7'] != ' ': # 대각선 3,5,7이 채워졌을 경우
                visual_Board(boarder)
                print("\n게임 종료.\n")                
                print(" ---------- " +turn + "가 승리했습니다. -----------")
                break 

        if count == 9: # x,y 모두 이기지 않고 보드가 꽉채워있으면 동점으로 선언
            print("\n게임 종료.\n")                
            print("동점입니다")


        if turn =='X': # 움직임이 있을때마다 플레이어 변경
            turn = 'Y'
        else:
            turn = 'X'        

if __name__ == "__main__":
    game()

X|Y|X
-+-+-
X|Y|Y
-+-+-
X|Y|X
당신 차례입니다,X. 어디로 이동할까요?
